<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/achatbot_zonos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install

In [ ]:
!cd /content && rm -rf achatbot && git clone https://github.com/ai-bot-pro/achatbot.git -b feat/tts

In [2]:
%cd /content/achatbot

/content/achatbot


In [29]:
!git submodule update --init --recursive

Submodule path 'deps/Zonos': checked out '43e4848b920003f4b9a9b427764ea33f4b56a59b'


In [ ]:
!git checkout . && git pull origin feat/tts

In [ ]:
!bash scripts/pypi_achatbot.sh dev

In [103]:
!pip install -q dist/achatbot-0.0.8.11-py3-none-any.whl[tts_zonos_hybrid,grpc]

In [ ]:
!apt install -y espeak-ng

# download ckpt


In [7]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')

In [8]:
!huggingface-cli login --token $HF_TOKEN --add-to-git-credential


Token is valid (permission: write).
The token `write` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `write`


In [9]:
!huggingface-cli download descript/dac_44khz --quie --local-dir /content/models/descript/dac_44khz


/content/models/descript/dac_44khz


In [25]:
!huggingface-cli download Zyphra/Zonos-v0.1-speaker-embedding --quie --local-dir /content/models/Zyphra/Zonos-v0.1-speaker-embedding


/content/models/Zyphra/Zonos-v0.1-speaker-embedding


In [10]:
!huggingface-cli download Zyphra/Zonos-v0.1-transformer --quie --local-dir /content/models/Zyphra/Zonos-v0.1-transformer


/content/models/Zyphra/Zonos-v0.1-transformer


In [11]:
!huggingface-cli download Zyphra/Zonos-v0.1-hybrid --quie --local-dir /content/models/Zyphra/Zonos-v0.1-hybrid


/content/models/Zyphra/Zonos-v0.1-hybrid


# demo

In [4]:
%cd /content/achatbot

/content/achatbot


In [5]:
import os
import sys
sys.path.insert(1, "deps/Zonos")
# NOTE: change deps/Zonos code; need restart ipynb!!!

In [6]:
import torch

def print_model_params(model: torch.nn.Module, extra_info=""):
    # print the number of parameters in the model
    model_million_params = sum(p.numel() for p in model.parameters()) / 1e6
    print(model)
    print(f"{extra_info} {model_million_params} M parameters")

In [7]:
import torch
import torchaudio
from deps.Zonos.zonos.model import Zonos
from deps.Zonos.zonos.conditioning import make_cond_dict


In [8]:
# need pip install achatbot[tts_zonos]
repo_id = "Zyphra/Zonos-v0.1-transformer"
model = Zonos.from_pretrained(repo_id, device="cuda",local_dir="/content/models/Zyphra/Zonos-v0.1-transformer",dac_model_path="/content/models/descript/dac_44khz").eval()
print_model_params(model)

Zonos(
  (backbone): TorchZonosBackbone(
    (layers): ModuleList(
      (0-25): 26 x TransformerBlock(
        (norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mixer): Attention(
          (in_proj): Linear(in_features=2048, out_features=3072, bias=False)
          (out_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (norm2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): FeedForward(
          (fc1): Linear(in_features=2048, out_features=16384, bias=False)
          (fc2): Linear(in_features=8192, out_features=2048, bias=False)
        )
      )
    )
    (norm_f): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
  )
  (prefix_conditioner): PrefixConditioner(
    (project): Linear(in_features=2048, out_features=2048, bias=True)
    (conditioners): ModuleList(
      (0): EspeakPhonemeConditioner(
        (project): Identity()
        (phoneme_embedder): Embedding(189, 2048)
      )
      (1): Passthro

In [9]:
# need pip install achatbot[tts_zonos_hybrid]
repo_id = "Zyphra/Zonos-v0.1-hybrid"
model = Zonos.from_pretrained(repo_id, device="cuda",local_dir="/content/models/Zyphra/Zonos-v0.1-hybrid",dac_model_path="/content/models/descript/dac_44khz").eval()
print_model_params(model)

Zonos(
  (backbone): MambaSSMZonosBackbone(
    (layers): ModuleList(
      (0): Block(
        (norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mixer): MHA(
          (rotary_emb): RotaryEmbedding()
          (in_proj): Linear(in_features=2048, out_features=3072, bias=False)
          (out_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (norm2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GatedMLP(
          (fc1): Linear(in_features=2048, out_features=16384, bias=False)
          (fc2): Linear(in_features=8192, out_features=2048, bias=False)
        )
      )
      (1-3): 3 x Block(
        (norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mixer): Mamba2(
          (in_proj): Linear(in_features=2048, out_features=8512, bias=False)
          (conv1d): Conv1d(4352, 4352, kernel_size=(4,), stride=(1,), padding=(3,), groups=4352)
          (act): SiLU()
          (norm): RMSNorm()
    

In [10]:
cond_dict = make_cond_dict(text="Hello, world!", speaker=None, language="en-us")
conditioning = model.prepare_conditioning(cond_dict)

codes = model.generate(conditioning)

wavs = model.autoencoder.decode(codes).cpu()
torchaudio.save("sample.wav", wavs[0], model.autoencoder.sampling_rate)

Generating:   3%|▎         | 85/2588 [00:01<00:56, 44.01it/s]


In [11]:
from IPython.display import Audio
Audio("sample.wav",autoplay=True)

# test tts

In [14]:
!ZONOS_LM_CHECKPOINT_DIR=/content/models/Zyphra/Zonos-v0.1-transformer \
  ZONOS_DAC_MODEL_DIR=/content/models/descript/dac_44khz \
  python -m unittest test.modules.speech.tts.test_zonos.TestZonosTTS.test_get_voices

2025-02-17 16:26:12,695 - torio._extension.utils - DEBUG - /usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py:114 - _find_ffmpeg_extension - Loading FFmpeg6
2025-02-17 16:26:12,696 - torio._extension.utils - DEBUG - /usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py:120 - _find_ffmpeg_extension - Failed to load FFmpeg6 extension.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py", line 116, in _find_ffmpeg_extension
    ext = _find_versionsed_ffmpeg_extension(ffmpeg_ver)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py", line 108, in _find_versionsed_ffmpeg_extension
    _load_lib(lib)
  File "/usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py", line 94, in _load_lib
    torch.ops.load_library(path)
  File "/usr/local/lib/python3.11/dist-packages/torch/_ops.py", line 1350, in load_library
    ctypes.CDLL

In [15]:
!ZONOS_LM_CHECKPOINT_DIR=/content/models/Zyphra/Zonos-v0.1-transformer \
  ZONOS_DAC_MODEL_DIR=/content/models/descript/dac_44khz \
  python -m unittest test.modules.speech.tts.test_zonos.TestZonosTTS.test_set_voice

2025-02-17 16:28:26,058 - torio._extension.utils - DEBUG - /usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py:114 - _find_ffmpeg_extension - Loading FFmpeg6
2025-02-17 16:28:26,059 - torio._extension.utils - DEBUG - /usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py:120 - _find_ffmpeg_extension - Failed to load FFmpeg6 extension.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py", line 116, in _find_ffmpeg_extension
    ext = _find_versionsed_ffmpeg_extension(ffmpeg_ver)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py", line 108, in _find_versionsed_ffmpeg_extension
    _load_lib(lib)
  File "/usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py", line 94, in _load_lib
    torch.ops.load_library(path)
  File "/usr/local/lib/python3.11/dist-packages/torch/_ops.py", line 1350, in load_library
    ctypes.CDLL

In [16]:
!ZONOS_LM_CHECKPOINT_DIR=/content/models/Zyphra/Zonos-v0.1-transformer \
  ZONOS_DAC_MODEL_DIR=/content/models/descript/dac_44khz \
  python -m unittest test.modules.speech.tts.test_zonos.TestZonosTTS.test_synthesize

2025-02-17 16:29:36,885 - torio._extension.utils - DEBUG - /usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py:114 - _find_ffmpeg_extension - Loading FFmpeg6
2025-02-17 16:29:36,886 - torio._extension.utils - DEBUG - /usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py:120 - _find_ffmpeg_extension - Failed to load FFmpeg6 extension.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py", line 116, in _find_ffmpeg_extension
    ext = _find_versionsed_ffmpeg_extension(ffmpeg_ver)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py", line 108, in _find_versionsed_ffmpeg_extension
    _load_lib(lib)
  File "/usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py", line 94, in _load_lib
    torch.ops.load_library(path)
  File "/usr/local/lib/python3.11/dist-packages/torch/_ops.py", line 1350, in load_library
    ctypes.CDLL

In [17]:
from IPython.display import Audio
Audio("/content/achatbot/records/test_tts_zonos.wav",autoplay=True)

In [18]:
!ZONOS_LM_CHECKPOINT_DIR=/content/models/Zyphra/Zonos-v0.1-transformer \
  ZONOS_DAC_MODEL_DIR=/content/models/descript/dac_44khz \
  ZONOS_REF_AUDIO_PATH=./test/audio_files/asr_example_zh.wav \
  python -m unittest test.modules.speech.tts.test_zonos.TestZonosTTS.test_synthesize

2025-02-17 16:31:40,495 - torio._extension.utils - DEBUG - /usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py:114 - _find_ffmpeg_extension - Loading FFmpeg6
2025-02-17 16:31:40,496 - torio._extension.utils - DEBUG - /usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py:120 - _find_ffmpeg_extension - Failed to load FFmpeg6 extension.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py", line 116, in _find_ffmpeg_extension
    ext = _find_versionsed_ffmpeg_extension(ffmpeg_ver)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py", line 108, in _find_versionsed_ffmpeg_extension
    _load_lib(lib)
  File "/usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py", line 94, in _load_lib
    torch.ops.load_library(path)
  File "/usr/local/lib/python3.11/dist-packages/torch/_ops.py", line 1350, in load_library
    ctypes.CDLL

In [19]:
from IPython.display import Audio
Audio("/content/achatbot/records/test_tts_zonos.wav",autoplay=True)

In [20]:
!ZONOS_LM_CHECKPOINT_DIR=/content/models/Zyphra/Zonos-v0.1-hybrid \
  ZONOS_DAC_MODEL_DIR=/content/models/descript/dac_44khz \
  python -m unittest test.modules.speech.tts.test_zonos.TestZonosTTS.test_synthesize

2025-02-17 16:37:04,762 - torio._extension.utils - DEBUG - /usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py:114 - _find_ffmpeg_extension - Loading FFmpeg6
2025-02-17 16:37:04,762 - torio._extension.utils - DEBUG - /usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py:120 - _find_ffmpeg_extension - Failed to load FFmpeg6 extension.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py", line 116, in _find_ffmpeg_extension
    ext = _find_versionsed_ffmpeg_extension(ffmpeg_ver)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py", line 108, in _find_versionsed_ffmpeg_extension
    _load_lib(lib)
  File "/usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py", line 94, in _load_lib
    torch.ops.load_library(path)
  File "/usr/local/lib/python3.11/dist-packages/torch/_ops.py", line 1350, in load_library
    ctypes.CDLL

In [21]:
from IPython.display import Audio
Audio("/content/achatbot/records/test_tts_zonos.wav",autoplay=True)

In [22]:
!ZONOS_LM_CHECKPOINT_DIR=/content/models/Zyphra/Zonos-v0.1-hybrid \
  ZONOS_DAC_MODEL_DIR=/content/models/descript/dac_44khz \
  ZONOS_REF_AUDIO_PATH=./test/audio_files/asr_example_zh.wav \
  python -m unittest test.modules.speech.tts.test_zonos.TestZonosTTS.test_synthesize

2025-02-17 16:37:46,865 - torio._extension.utils - DEBUG - /usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py:114 - _find_ffmpeg_extension - Loading FFmpeg6
2025-02-17 16:37:46,866 - torio._extension.utils - DEBUG - /usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py:120 - _find_ffmpeg_extension - Failed to load FFmpeg6 extension.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py", line 116, in _find_ffmpeg_extension
    ext = _find_versionsed_ffmpeg_extension(ffmpeg_ver)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py", line 108, in _find_versionsed_ffmpeg_extension
    _load_lib(lib)
  File "/usr/local/lib/python3.11/dist-packages/torio/_extension/utils.py", line 94, in _load_lib
    torch.ops.load_library(path)
  File "/usr/local/lib/python3.11/dist-packages/torch/_ops.py", line 1350, in load_library
    ctypes.CDLL

In [23]:
from IPython.display import Audio
Audio("/content/achatbot/records/test_tts_zonos.wav",autoplay=True)

# tts speaker with grpc


In [2]:
%cd /content/achatbot

/content/achatbot


In [ ]:
#!python -m src.cmd.grpc.speaker.server.serve

In [3]:
!TTS_TAG=tts_zonos \
    ZONOS_LM_CHECKPOINT_DIR=/content/models/Zyphra/Zonos-v0.1-transformer \
    ZONOS_DAC_MODEL_DIR=/content/models/descript/dac_44khz \
    SPEAKER_EMBEDDING_MODEL_DIR=/content/models/Zyphra/Zonos-v0.1-speaker-embedding \
    ZONOS_REF_AUDIO_PATH=./test/audio_files/asr_example_zh.wav \
    IS_SAVE=1 IS_RELOAD=1 python -m src.cmd.grpc.speaker.client

2025-02-17 16:49:00,843 - chat-bot-tts-client - DEBUG - /content/achatbot/src/cmd/grpc/speaker/client.py:59 - load_model - tts_tag: "tts_zonos"
is_reload: true
json_kwargs: "{\"device\": null, \"lm_checkpoint_dir\": \"/content/models/Zyphra/Zonos-v0.1-transformer\", \"dac_model_dir\": \"/content/models/descript/dac_44khz\", \"speaker_embedding_model_dir\": \"/content/models/Zyphra/Zonos-v0.1-speaker-embedding\", \"ref_audio_file_path\": \"./test/audio_files/asr_example_zh.wav\", \"prompt_text\": \"\", \"is_save\": false, \"output_codebook_indices_dir\": \"/content/achatbot/models/zonos_codebook_indices\", \"seed\": 42, \"chunk_size\": 40, \"warmup_steps\": 1, \"warm_up_text\": \"hello world\", \"language\": \"en-us\", \"emotion\": [0.3077, 0.0256, 0.0256, 0.0256, 0.0256, 0.0256, 0.2564, 0.3077], \"fmax\": 22050.0, \"pitch_std\": 20.0, \"speaking_rate\": 15.0, \"vqscore_8\": [0.78, 0.78, 0.78, 0.78, 0.78, 0.78, 0.78, 0.78], \"ctc_loss\": 0.0, \"dnsmos_ovrl\": 4.0, \"tts_stream\": true, 

In [4]:
from IPython.display import Audio
Audio("/content/achatbot/records/grpc_tts_zonos.wav",autoplay=True)
